In [2]:
!pip install pyspark

In [35]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [36]:
spark = SparkSession.builder.appName('Welmart_data_analysis').getOrCreate()

In [37]:
csv_file = "/Welmart-Data.csv"
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(csv_file)

In [38]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [15]:
# 1. Determine the best-selling product sub-category
best_selling_product = df.groupBy("Sub-Category").agg(sum("Sales").alias("Total Sales")).orderBy(desc("Total Sales")).limit(1)
best_selling_product.show()

+------------+-----------------+
|Sub-Category|      Total Sales|
+------------+-----------------+
|      Phones|329753.0880000001|
+------------+-----------------+



In [17]:
# 2. Identify the product category generating the highest revenue
highest_revenue_category = df.groupBy("Category").agg(sum("Sales").alias("Total Revenue")).orderBy(desc("Total Revenue")).limit(1)
highest_revenue_category.show()

+----------+-----------------+
|  Category|    Total Revenue|
+----------+-----------------+
|Technology|835900.0669999964|
+----------+-----------------+



In [19]:
# 3. Compile a top 10 list of the most valuable customers.
top_10_customers = df.groupBy("Customer Name").agg(sum("Sales").alias("Total Revenue")).orderBy(desc("Total Revenue")).limit(10)
top_10_customers.show()

+------------------+------------------+
|     Customer Name|     Total Revenue|
+------------------+------------------+
|       Sean Miller|          25043.05|
|      Tamara Chand|19017.847999999998|
|      Raymond Buch|         15117.339|
|      Tom Ashbrook|          14595.62|
|     Adrian Barton|14355.610999999997|
|      Sanjit Chand|14142.333999999999|
|      Ken Lonsdale|         14071.917|
|      Hunter Lopez|12873.297999999999|
|      Sanjit Engle|12209.438000000002|
|Christopher Conant|         12129.072|
+------------------+------------------+



In [22]:
# 4. Determine the state responsible for the highest number of orders.
highest_order_state = df.groupBy("State").agg(count("Order ID").alias("total orders")).orderBy(desc("total orders")).limit(1)
highest_order_state.show()

+----------+------------+
|     State|total orders|
+----------+------------+
|California|        2001|
+----------+------------+



In [48]:
# 5. Find the year with the highest revenue generation.

order_year = df.withColumn("Order Date", to_date(df["Order Date"], "M/d/yyyy"))
order_year = order_year.withColumn("order_year", year(order_year["Order Date"]))
great_year = order_year.groupBy("order_year").agg(count("Sales").alias("total Revenue")).orderBy(desc("total Revenue")).limit(1)
great_year.show()

+----------+-------------+
|order_year|total Revenue|
+----------+-------------+
|      2017|         3312|
+----------+-------------+

